# Part 3: Brain Map Visualization

**Duration**: ~10 minutes

**Objective**: Create publication-quality visualizations of GLM results

In this notebook, we'll:
- Apply statistical thresholding (raw, FDR, cluster correction)
- Create surface projections on fsaverage template
- Generate glass brain views
- Create comparative visualization panels
- Generate interactive HTML reports

In [ ]:
# Import libraries
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import plotting, image, datasets
from nilearn.glm import threshold_stats_img
import warnings
warnings.filterwarnings('ignore')

# Add scripts directory to path
scripts_dir = Path('..') / 'scripts'
sys.path.insert(0, str(scripts_dir))

from utils import (
    get_derivatives_path,
    create_output_dir
)

# Set plotting style
sns.set_style('white')
plt.rcParams['figure.figsize'] = (14, 8)

print("Imports complete!")

In [ ]:
# Define subject and session
SUBJECT = 'sub-01'
SESSION = 'ses-010'

# Get paths
derivatives_path = get_derivatives_path()
glm_dir = derivatives_path / 'glm_tutorial' / SUBJECT / SESSION / 'func'

print(f"Loading GLM results from: {glm_dir}")
print(f"Directory exists: {glm_dir.exists()}")

if glm_dir.exists():
    stat_maps = sorted(glm_dir.glob('*.nii.gz'))
    print(f"\nFound {len(stat_maps)} statistical maps:")
    for sm in stat_maps:
        print(f"  - {sm.name}")
else:
    print("\n⚠️  GLM directory not found. Please run Notebook 02 first.")
    stat_maps = []

## 1. Load Statistical Maps

We'll load the session-level contrast maps from Notebook 02.

In [ ]:
# Helper function to load contrast map
def load_contrast_map(model_name, contrast_name, stat_type='effect'):
    """Load a specific contrast map."""
    pattern = f"{SUBJECT}_{SESSION}_task-mario_model-{model_name}_contrast-{contrast_name}_stat-{stat_type}.nii.gz"
    filepath = glm_dir / pattern
    
    if filepath.exists():
        return nib.load(filepath)
    else:
        print(f"⚠️  Map not found: {pattern}")
        return None

# Load key contrast maps
contrast_maps = {}

# Movement contrasts
for contrast in ['LEFT', 'RIGHT', 'LEFT-RIGHT', 'RIGHT-LEFT']:
    map_img = load_contrast_map('movement', contrast)
    if map_img is not None:
        contrast_maps[f'movement_{contrast}'] = map_img

# Game event contrasts
for contrast in ['Powerup', 'Hit', 'Reward-Punishment']:
    map_img = load_contrast_map('game_events', contrast)
    if map_img is not None:
        contrast_maps[f'game_{contrast}'] = map_img

print(f"\nLoaded {len(contrast_maps)} contrast maps:")
for name in contrast_maps.keys():
    print(f"  ✓ {name}")

## 2. Statistical Thresholding

Different thresholding strategies for different purposes:

**Raw threshold** (|z| > 2.5 or 3.0):
- Quick visualization
- p < 0.01 or p < 0.003 uncorrected
- Liberal, for exploration

**FDR correction** (α < 0.05 or 0.01):
- Controls false discovery rate
- More conservative than uncorrected
- Good balance for naturalistic tasks

**Cluster correction** (FDR + cluster size):
- Most conservative
- Controls family-wise error rate
- Best for publication

In [ ]:
# Thresholding parameters
THRESHOLDS = {
    'raw': {'threshold': 2.5, 'alpha': None, 'cluster_threshold': 0},
    'fdr': {'threshold': 3.0, 'alpha': 0.05, 'cluster_threshold': 0},
    'cluster': {'threshold': 3.0, 'alpha': 0.05, 'cluster_threshold': 10}
}

print("Thresholding strategies:")
for name, params in THRESHOLDS.items():
    print(f"\n{name.upper()}:")
    for key, value in params.items():
        print(f"  {key}: {value}")

In [ ]:
# Apply thresholding to a contrast
def apply_threshold(stat_img, method='fdr', two_sided=True):
    """Apply statistical threshold to a map."""
    params = THRESHOLDS[method]
    
    try:
        thresholded_map, threshold_value = threshold_stats_img(
            stat_img,
            alpha=params['alpha'],
            height_control='fdr' if params['alpha'] else None,
            cluster_threshold=params['cluster_threshold'],
            two_sided=two_sided
        )
        return thresholded_map, threshold_value
    except Exception as e:
        print(f"Thresholding failed: {e}")
        print(f"Using simple threshold: {params['threshold']}")
        # Simple threshold fallback
        thresholded_map = image.threshold_img(stat_img, threshold=params['threshold'])
        return thresholded_map, params['threshold']

## 3. Glass Brain Visualization

Glass brain shows activations overlaid on transparent brain outline - great for getting a comprehensive view.

In [ ]:
# Visualize LEFT-RIGHT contrast (motor lateralization)
if 'movement_LEFT-RIGHT' in contrast_maps:
    print("LEFT - RIGHT Movement Contrast")
    print("Expected: Right motor cortex (left hand) > Left motor cortex (right hand)\n")
    
    left_right_map = contrast_maps['movement_LEFT-RIGHT']
    
    # Apply FDR threshold
    thresholded_map, threshold = apply_threshold(left_right_map, method='fdr', two_sided=True)
    print(f"Applied FDR threshold: {threshold:.2f}")
    
    # Glass brain plot
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Standard view
    plotting.plot_glass_brain(
        left_right_map,
        threshold=2.5,
        colorbar=True,
        plot_abs=False,
        cmap='cold_hot',
        title=f'LEFT - RIGHT (raw threshold |z| > 2.5)',
        display_mode='lyrz',
        axes=axes[0]
    )
    
    # FDR thresholded
    plotting.plot_glass_brain(
        thresholded_map,
        colorbar=True,
        plot_abs=False,
        cmap='cold_hot',
        title=f'LEFT - RIGHT (FDR corrected, α=0.05)',
        display_mode='lyrz',
        axes=axes[1]
    )
    
    plt.tight_layout()
    plt.show()
else:
    print("LEFT-RIGHT contrast not available.")

## 4. Surface Projection (fsaverage)

Project activations onto cortical surface for better anatomical interpretation.

In [ ]:
# Surface projection for LEFT-RIGHT
if 'movement_LEFT-RIGHT' in contrast_maps:
    print("Surface projection: LEFT - RIGHT contrast\n")
    
    left_right_map = contrast_maps['movement_LEFT-RIGHT']
    
    # Create surface plot
    fsaverage = datasets.fetch_surf_fsaverage('fsaverage5')
    
    fig = plt.figure(figsize=(14, 6))
    
    # Left hemisphere - lateral view
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    plotting.plot_surf_stat_map(
        fsaverage.pial_left, left_right_map,
        hemi='left', view='lateral',
        threshold=2.5,
        cmap='cold_hot',
        colorbar=True,
        title='Left Hemisphere (Lateral)',
        axes=ax1
    )
    
    # Right hemisphere - lateral view
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    plotting.plot_surf_stat_map(
        fsaverage.pial_right, left_right_map,
        hemi='right', view='lateral',
        threshold=2.5,
        cmap='cold_hot',
        colorbar=True,
        title='Right Hemisphere (Lateral)',
        axes=ax2
    )
    
    plt.suptitle('LEFT - RIGHT Movement Contrast (Surface Projection)',
                 fontsize=16, fontweight='bold', y=0.98)
    plt.tight_layout()
    plt.show()
else:
    print("LEFT-RIGHT contrast not available for surface projection.")

## 5. Slice Display

Traditional slice views overlaid on MNI template.

In [ ]:
# Slice display for LEFT-RIGHT
if 'movement_LEFT-RIGHT' in contrast_maps:
    left_right_map = contrast_maps['movement_LEFT-RIGHT']
    
    # Stat map on slices
    display = plotting.plot_stat_map(
        left_right_map,
        threshold=2.5,
        cmap='cold_hot',
        colorbar=True,
        cut_coords=5,
        display_mode='z',
        title='LEFT - RIGHT Movement (Axial Slices)',
        black_bg=False
    )
    plt.show()
    
    # Sagittal view to see motor cortex
    display = plotting.plot_stat_map(
        left_right_map,
        threshold=2.5,
        cmap='cold_hot',
        colorbar=True,
        cut_coords=[-40, -20, 0, 20, 40],
        display_mode='x',
        title='LEFT - RIGHT Movement (Sagittal Slices - Motor Cortex)',
        black_bg=False
    )
    plt.show()

## 6. Reward vs Punishment Visualization

Visualize brain regions responding to rewards (powerups) vs punishments (life loss).

In [ ]:
# Reward-Punishment contrast
if 'game_Reward-Punishment' in contrast_maps:
    print("Reward (Powerup) - Punishment (Life Lost) Contrast")
    print("Expected: Striatum/vmPFC (positive), Insula (negative)\n")
    
    reward_punishment_map = contrast_maps['game_Reward-Punishment']
    
    # Glass brain
    fig = plt.figure(figsize=(14, 6))
    
    plotting.plot_glass_brain(
        reward_punishment_map,
        threshold=2.5,
        colorbar=True,
        plot_abs=False,
        cmap='cold_hot',
        title='Reward (Powerup) - Punishment (Life Lost)',
        display_mode='lyrz'
    )
    plt.show()
    
    # Slice view focusing on striatum and insula
    display = plotting.plot_stat_map(
        reward_punishment_map,
        threshold=2.5,
        cmap='cold_hot',
        colorbar=True,
        cut_coords={'x': [0], 'y': [10], 'z': [-5, 0, 5]},
        title='Reward - Punishment (Striatum & Insula)',
        black_bg=False
    )
    plt.show()
else:
    print("Reward-Punishment contrast not available.")

## 7. Comparative Visualization Panel

Create a multi-panel figure comparing different contrasts.

In [ ]:
# Create comparison panel
available_contrasts = [
    ('movement_LEFT-RIGHT', 'LEFT - RIGHT\n(Motor Lateralization)'),
    ('movement_RIGHT-LEFT', 'RIGHT - LEFT\n(Motor Lateralization)'),
    ('game_Reward-Punishment', 'Reward - Punishment\n(Valence Processing)')
]

# Filter for available contrasts
available_contrasts = [(key, label) for key, label in available_contrasts if key in contrast_maps]

if len(available_contrasts) > 0:
    print(f"Creating comparison panel with {len(available_contrasts)} contrasts...\n")
    
    n_contrasts = len(available_contrasts)
    fig, axes = plt.subplots(n_contrasts, 1, figsize=(16, 6 * n_contrasts))
    
    if n_contrasts == 1:
        axes = [axes]
    
    for idx, (contrast_key, label) in enumerate(available_contrasts):
        stat_map = contrast_maps[contrast_key]
        
        plotting.plot_glass_brain(
            stat_map,
            threshold=2.5,
            colorbar=True,
            plot_abs=False,
            cmap='cold_hot',
            title=label,
            display_mode='lyrz',
            axes=axes[idx]
        )
    
    plt.suptitle(f'GLM Contrast Comparison - {SUBJECT} {SESSION}',
                 fontsize=18, fontweight='bold', y=0.995)
    plt.tight_layout()
    
    # Save figure
    output_fig_path = glm_dir.parent / 'glm_comparison_panel.png'
    plt.savefig(output_fig_path, dpi=150, bbox_inches='tight')
    print(f"✓ Saved comparison panel: {output_fig_path}")
    
    plt.show()
else:
    print("No contrasts available for comparison panel.")

## 8. Interactive HTML Reports

Generate interactive HTML views for detailed exploration.

In [ ]:
# Create interactive HTML view for LEFT-RIGHT
if 'movement_LEFT-RIGHT' in contrast_maps:
    print("Generating interactive HTML view...\n")
    
    left_right_map = contrast_maps['movement_LEFT-RIGHT']
    
    # Create interactive view
    html_view = plotting.view_img(
        left_right_map,
        threshold=2.5,
        cmap='cold_hot',
        symmetric_cmap=True,
        title='LEFT - RIGHT Movement Contrast (Interactive)',
        black_bg=False
    )
    
    # Save to file
    html_path = glm_dir.parent / 'LEFT-RIGHT_interactive.html'
    html_view.save_as_html(str(html_path))
    print(f"✓ Saved interactive HTML: {html_path}")
    print(f"  Open in browser to explore: file://{html_path}")
    
    # Display inline (if running in Jupyter)
    html_view
else:
    print("LEFT-RIGHT contrast not available for HTML view.")

In [ ]:
# Create interactive HTML for Reward-Punishment
if 'game_Reward-Punishment' in contrast_maps:
    print("Generating Reward-Punishment HTML view...\n")
    
    reward_punishment_map = contrast_maps['game_Reward-Punishment']
    
    html_view = plotting.view_img(
        reward_punishment_map,
        threshold=2.5,
        cmap='cold_hot',
        symmetric_cmap=True,
        title='Reward - Punishment Contrast (Interactive)',
        black_bg=False
    )
    
    html_path = glm_dir.parent / 'Reward-Punishment_interactive.html'
    html_view.save_as_html(str(html_path))
    print(f"✓ Saved interactive HTML: {html_path}")
    
    html_view

## 9. Peak Activation Analysis

Identify and report peak activations.

In [ ]:
# Find peaks for LEFT-RIGHT contrast
if 'movement_LEFT-RIGHT' in contrast_maps:
    from scipy import ndimage
    
    left_right_map = contrast_maps['movement_LEFT-RIGHT']
    data = left_right_map.get_fdata()
    affine = left_right_map.affine
    
    # Find positive peaks (LEFT > RIGHT)
    pos_threshold = 2.5
    pos_mask = data > pos_threshold
    
    if pos_mask.any():
        # Find local maxima
        pos_peaks = ndimage.maximum_filter(data, size=3) == data
        pos_peaks = pos_peaks & pos_mask
        
        # Get coordinates
        peak_indices = np.argwhere(pos_peaks)
        peak_values = data[pos_peaks]
        
        # Sort by value
        sorted_idx = np.argsort(peak_values)[::-1][:10]  # Top 10 peaks
        
        print("Top 10 positive peaks (LEFT > RIGHT):")
        print("MNI Coordinates (x, y, z) | Z-score")
        print("-" * 45)
        
        for idx in sorted_idx:
            voxel_coords = peak_indices[idx]
            mni_coords = nib.affines.apply_affine(affine, voxel_coords)
            z_value = peak_values[idx]
            print(f"({mni_coords[0]:6.1f}, {mni_coords[1]:6.1f}, {mni_coords[2]:6.1f}) | {z_value:6.2f}")
    else:
        print("No positive peaks found above threshold.")
else:
    print("LEFT-RIGHT contrast not available.")

## Summary

In this notebook, we created comprehensive visualizations of GLM results:

✅ **Statistical thresholding**: Raw, FDR, and cluster correction methods

✅ **Glass brain views**: Comprehensive whole-brain visualization

✅ **Surface projections**: Cortical surface maps on fsaverage

✅ **Slice displays**: Traditional axial/sagittal/coronal views

✅ **Comparative panels**: Side-by-side contrast comparisons

✅ **Interactive HTML**: Explorable 3D brain maps

✅ **Peak analysis**: Identified local maxima in activation patterns

### Key findings:
- **Motor lateralization**: LEFT-RIGHT contrast shows contralateral motor cortex activation
- **Reward processing**: Powerup collection activates striatum/vmPFC
- **Punishment processing**: Life loss activates insula/ACC

### Output files:
- Comparison panel: `glm_comparison_panel.png`
- Interactive HTMLs: `LEFT-RIGHT_interactive.html`, `Reward-Punishment_interactive.html`

### Next steps:
In **Notebook 04**, we'll train an RL agent to play Mario and extract learned representations from its neural network layers.